# Data Exploration

In [ ]:
!pip install langchain==0.0.340 --quiet
!pip install chromadb==0.4.13 --quiet
!pip install rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.8/437.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.to

In [ ]:
import pandas as pd
import json
from pathlib import Path
from pprint import pprint
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# extracted_df = pd.read_csv('processed_df.csv')
extracted_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/chunked_data_10000.csv')
extracted_df['sections'] = [i.lower() for i in extracted_df['sections']]

KeyError: 'sections'

In [ ]:
extracted_df['sections'][0]

'ai-driven security operations (soc) | fortinet , , security operations:  , detect and disrupt, then investigate and remediate:  the fortinet security operations (secops) platform seamlessly integrates behavior-based sensors to detect and disrupt threat actors across the attack surface and along the cyber kill chain. backed by our operating system, fortios, fortinet delivers centralized investigation and remediation that can be orchestrated, automated, and/or augmented to reduce cyber risk, cost, and operational effort., the fortinet security operations (secops) platform seamlessly integrates behavior-based sensors to detect and disrupt threat actors across the attack surface and along the cyber kill chain. backed by our operating system, fortios, fortinet delivers centralized investigation and remediation that can be orchestrated, automated, and/or augmented to reduce cyber risk, cost, and operational effort., early detection and prevention:  the fortinet secops platform offers the br

In [ ]:
def clean(x):
    x = x.replace('[','')
    x = x.replace("'",'')
    x = x.replace(']','')
    x = x.replace('xa0','')
    x = x.replace('{content:',"")
    x = x.replace('{','')
    x = x.replace('}','')
    x = x.replace('\n',' ,')
    x = x.replace("\\",' ')
    return x
extracted_df['sections'] = [clean(i) for i in extracted_df['sections']]

# Chunking

In [ ]:
extracted_df.head()

,url,sections
0,https://www.fortinet.com/fortiguard/threat-int...,ai-driven security operations (soc) | fortinet...
1,https://www.fortinet.com/products/fortipoints/,"flexible consumption licensing | fortipoints ,..."
2,https://www.fortinet.com/products/sandbox/,"fortiguard sandbox service and products , , fo..."
3,https://www.fortinet.com/corporate/about-us/ev...,"upcoming webinars, events and conferences arou..."
4,https://www.fortinet.com/products/network-acce...,premier network access control (nac) solutions...


In [ ]:
def split_text(input_data,separator = ',',chunk_size = 500):
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, separator=separator,chunk_overlap = 50)
    docs, metadatas = [], []
    for page in input_data:
        binary_split = text_splitter.split_text(page['text'])
        docs.extend(binary_split)
        metadatas.extend([{"source": page['source']}] * len(binary_split))
    return metadatas,docs

input_data = []
for i in range(0,len(extracted_df)):
    temp = {'text': extracted_df["sections"][i], 'source': extracted_df["url"][i]}
    input_data.append(temp)
# metadatas,docs = split_text(input_data)
len(docs),len(metadatas)

Created a chunk of size 535, which is longer than the specified 500
Created a chunk of size 591, which is longer than the specified 500
Created a chunk of size 630, which is longer than the specified 500
Created a chunk of size 523, which is longer than the specified 500
Created a chunk of size 535, which is longer than the specified 500
Created a chunk of size 656, which is longer than the specified 500
Created a chunk of size 535, which is longer than the specified 500
Created a chunk of size 523, which is longer than the specified 500
Created a chunk of size 514, which is longer than the specified 500
Created a chunk of size 532, which is longer than the specified 500
Created a chunk of size 547, which is longer than the specified 500
Created a chunk of size 578, which is longer than the specified 500
Created a chunk of size 549, which is longer than the specified 500
Created a chunk of size 507, which is longer than the specified 500
Created a chunk of size 602, which is longer tha

(145716, 145716)

In [ ]:
df = pd.DataFrame()
df['chunks']=docs
df['len']=[len(i) for i in docs]
df['ref_link'] = metadatas

df_unchunked= df[df['len']>500]
df_chunked= df[df['len']<=500]

df_unchunked.shape

(1299, 3)

In [ ]:
input_data_remain_chunk = []
for i in df_unchunked.values :
    temp = {'text': i[0], 'source': i[2]}
    input_data_remain_chunk.append(temp)
metadatas,docs = split_text(input_data_remain_chunk,separator = '.')
len(docs),len(metadatas)

Created a chunk of size 511, which is longer than the specified 500
Created a chunk of size 755, which is longer than the specified 500
Created a chunk of size 755, which is longer than the specified 500
Created a chunk of size 755, which is longer than the specified 500
Created a chunk of size 755, which is longer than the specified 500
Created a chunk of size 755, which is longer than the specified 500
Created a chunk of size 511, which is longer than the specified 500
Created a chunk of size 1789, which is longer than the specified 500
Created a chunk of size 1287, which is longer than the specified 500
Created a chunk of size 1156, which is longer than the specified 500
Created a chunk of size 1021, which is longer than the specified 500
Created a chunk of size 1568, which is longer than the specified 500
Created a chunk of size 963, which is longer than the specified 500
Created a chunk of size 1061, which is longer than the specified 500
Created a chunk of size 687, which is long

(2666, 2666)

In [ ]:
df = pd.DataFrame()
df['chunks']=docs
df['len']=[len(i) for i in docs]
df['ref_link'] = metadatas
# Concatenate DataFrame and row
df = pd.concat([df_chunked, df], ignore_index=True)
df.shape

(147083, 3)

In [ ]:
df_remaining_chunk= df[df['len']>500]
df_remaining_chunk.shape,len(input_data_remain_chunk)

((104, 3), 1299)

In [ ]:
df.to_csv('chunked_data.csv')